@author: Manal Rahal - 3 Nov 2022 <br>
@title: Models testing on the big dataset

In [ ]:
! pip install seqfold

In [ ]:
import os
mydir = os.getcwd() # would be the MAIN folder
mydir

In [ ]:
! python Encode_funcs_test.py

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime
from itertools import product, combinations
import sklearn.model_selection as skl_ms
import sklearn.metrics as skl_me
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import sklearn.svm as svm
import matplotlib.pyplot as plt 
from sklearn import preprocessing
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing
import pickle

In [ ]:
import Encode_funcs_test
from Encode_funcs_test import *
from Encode_funcs_test import encode_input

In [ ]:
import supporting_funcs
from supporting_funcs import get_null_percentage, isnullcolumns, isinfinitecolumns, find_outliers_IQR, generate_ID
from supporting_funcs import draw_scatter, is_lost_sulfur, count_sulfur, return_phosphated_seq, is_phosphated, percentage_phosphated

In [ ]:
! python supporting_funcs.py

# Loading duplicates

In [ ]:
#dups1 = pd.read_excel('G1_duplicates.xlsx')
#dups2 =pd.read_excel('G2_duplicates.xlsx')
#dups3 =pd.read_excel('G3_duplicates.xlsx')

In [ ]:
#dups1 = dups1['Sequence']
#dups2 = dups2['Sequence']
#dups3 = dups3['Sequence']

In [ ]:
#dups2

## **Loading and Encoding Retention Time Dataset**

In [ ]:
#__Loading data from excel file, each gradient is in a sheet

xls = pd.ExcelFile('../data/oligodata121022.xlsx')

In [ ]:
oligo_G22 =pd.read_excel(xls, sheet_name='22min gradient')

In [ ]:
oligo_G22.info()

In [ ]:
oligo_G22['Vinj'].unique()

In [ ]:
oligo_G22[oligo_G22['Vinj'] == 25].count()

#0.5 = 222
#1 = 3
#3 = 225
#15 = 166
#20 = 30
#25 = 240

## ---> Checking null and infinite values

In [ ]:
oligo_G22.info()

In [ ]:
oligo_G22 = oligo_G22.dropna(subset=['tR1'], how='all')

In [ ]:
oligo_G22.describe() #10 values in tR1 are missing, I decided not to replace them with tR2 values given that the quality of tR2 data is not of high quality

## Checking duplicates

In [ ]:
pd.set_option('display.max_rows', 500)

duplicates = oligo_G22[oligo_G22.duplicated(subset=['Sequence','Vinj'],keep='first')]
 
print("Duplicate Rows :")
 
duplicates 

In [ ]:
duplicates = duplicates['Sequence']

In [ ]:
oligo_G22_clean = oligo_G22[~oligo_G22['Sequence'].isin(duplicates)]
oligo_G22_clean.shape

In [ ]:
oligo_G22_clean = oligo_G22_clean.dropna(subset=['tR1'], how='all')

In [ ]:
input_data = oligo_G22_clean.loc[:,['Sequence', 'Vinj']]
print(input_data.shape)
input_data.head()

In [ ]:
print(isnullcolumns(input_data))

In [ ]:
input_data.info()

In [ ]:
input_data.head(5)

In [ ]:
target_data = oligo_G22_clean.loc[:,['tR1']]
print(target_data.shape)
target_data.head()

In [ ]:
null_values = target_data[target_data.isnull().any(axis=1)]
print (len(null_values))
print (null_values)

In [ ]:
target_data.head(5)

In [ ]:
target_data.info()

In [ ]:
input_data.info()

## Train Test split (RUN ONLY ONCE)

In [ ]:
train_ratio = 0.8
test_ratio = 0.2

x_train, x_test, y_train, y_test = train_test_split(input_data, target_data, test_size=test_ratio, random_state=42,shuffle=True)

print('xtrain ', x_train.shape,'\n')
print('ytrain ', y_train.shape,'\n')
print('xtest ', x_test.shape,'\n')
print('ytest ', y_test.shape,'\n')

In [ ]:
y_train

In [ ]:
#### RUN ONLY ONCE THEN READ THE DATASETS FROM THE EXPERTED FILES
x_train.to_excel("G22_xtrain.xlsx")
y_train.to_excel("G22_ytrain.xlsx")
x_test.to_excel("G22_xtest.xlsx")
y_test.to_excel("G22_ytest.xlsx")

## **Applying Machine Learning Models**

In [ ]:
features = {}

features['count'] = ['Fa', 'Ft', 'Fg', 'Fc', 'Length']
features ['sulfurcount'] = ['Fsulf']
features['contact'] = prod('ATGC', repeat = 2)
features['scontact'] = [f"{comb[0]}{comb[-1]}_{comb[-1]}{comb[0]}" for comb in pair_comb('ATGC', 2)]
features['position'] = ['first position', 'last position']

featvec = ['count','sulfurcount','contact','scontact','position']
featcombinations = []

for i in range(len(featvec)):
    for c in list(combinations(featvec, i + 1)):
        featcombinations.append(list(c))

featvec

In [ ]:
featcombinations

In [ ]:
x_train = pd.read_excel('G22_xtrain.xlsx')
y_train = pd.read_excel('G22_ytrain.xlsx')

In [ ]:
x_train.info()

In [ ]:
x_train.head()

In [ ]:
y_train.info()

In [ ]:
y_train

In [ ]:
print(isnullcolumns(y_train))

In [ ]:
df_sequence_input=x_train["Sequence"]
df_sequence_input.head()

In [ ]:
sequence =[]
for i,seq in enumerate (df_sequence_input):
    sequence=np.append(sequence, seq)

In [ ]:
len(sequence)

In [ ]:
df_tRvec = y_train['tR1']

In [ ]:
df_tRvec

In [ ]:
sequence.shape

In [ ]:
print(isnullcolumns(df_tRvec))

In [ ]:
df_tRvec.shape

In [ ]:
inj = x_train["Vinj"]
#inj = inj.reset_index()

In [ ]:
inj

### ---> Testing encode_input function after fixing the contact and scontact

In [ ]:
test_data = encode_input(sequence, featvec)
test_data = pd.concat([test_data, inj], axis=1)
test_data

In [ ]:
print(isnullcolumns(test_data))

### 1. Gradient boost

In [ ]:
params = {
            'max_depth': [5,10,15,20,50],
            'learning_rate': [0.001, 0.01, 0.1, 0.2],
            'n_estimators': [100, 500, 1000],
            'max_leaf_nodes': [2, 5, 10]
        }

In [ ]:
start = time.time()

#__defining variables
splits = 3
count = 0
best_perfomer_dict=dict()
scores = []

#__applying model to every feature combination, 31 features' combinations incl.  position and sulfur count features and excl. hairpin features

for j, featcomb in enumerate(featcombinations):
    inputvec_train = encode_input(sequence, featcomb)
    #inputvec_train['Vinj']=x_train["Vinj"]
    inputvec_train = pd.concat([inputvec_train, inj['Vinj']], axis=1)
    #print(inputvec_train)
    #print(isnullcolumns(inj['Vinj']))
    
    
    for col in  inputvec_train.columns[0:30]:
        inputvec_train[col] = pd.to_numeric(inputvec_train[col], errors='coerce')
#print (inputvec_train.info())
        
    xtrain, xvalid, ytrain, yvalid= train_test_split(inputvec_train, df_tRvec, test_size = 0.2, random_state = 42, shuffle=True)
        #print(len(X_train))
        #print(X_train)
        #print(X_test)
        
    model = GradientBoostingRegressor()
                
    grid = skl_ms.GridSearchCV(model, params, scoring = ['neg_root_mean_squared_error','r2'], refit = 'neg_root_mean_squared_error', cv = splits)
    grid.fit(xtrain, ytrain)
    model = grid.best_estimator_
                
    pre = model.predict(xvalid)
    err = skl_me.mean_squared_error(yvalid, pre, squared = False)
    count += 1
    print(f'\ncount: {count}')
            
    if ' '.join(featcomb) in best_perfomer_dict.keys():

        best_perfomer_dict[f' '.join(featcomb)] += err
            
    else:
            
        best_perfomer_dict[' '.join(featcomb)] = err
            

print(f'\n --- %s seconds --- % {time.time() - start}')

Min_error = min(best_perfomer_dict.values()) 
Min_key = min(best_perfomer_dict, key=best_perfomer_dict.get)  

In [ ]:
model = grid.best_estimator_
print ("grid.best_estimator_", grid.best_estimator_)
print ("grid.best_score_", grid.best_score_)
        
filename = f"GB_G2.sav"
        
pickle.dump(model, open(filename, 'wb'))

In [ ]:
print ('Grid Search and Features selection results:\n')
print (best_perfomer_dict)

In [ ]:
print(f"features {Min_key}")
print(f"RMSE: {Min_error}")
print(f'Best parameters: {grid.best_estimator_}')

In [ ]:
GB_results = pd.DataFrame(grid.cv_results_)
GB_results.to_excel("G2_GB_grid.xlsx")

In [ ]:
best_estim = grid.best_estimator_
best_estim

In [ ]:
df_tRvec

In [ ]:
best_featvec = ['count','sulfurcount', 'contact','position']

In [ ]:
alldata_train = encode_input(sequence, best_featvec)
alldata_train = pd.concat([alldata_train, inj['Vinj']], axis=1)

for col in  alldata_train.columns[0:30]:
    alldata_train[col] = pd.to_numeric(alldata_train[col], errors='coerce')

alldata_train

In [ ]:
best_estim.fit(alldata_train, df_tRvec)

In [ ]:
acc_train = round(best_estim.score(alldata_train, df_tRvec), 3)
print("training r2", acc_train)

In [ ]:
predtr = best_estim.predict(alldata_train)

rmsetr = skl_me.mean_squared_error(df_tRvec, predtr, squared = False)

print ('training rmse', rmsetr)

In [ ]:
x_test.head()

In [ ]:
x_test.shape

In [ ]:
inj_ts = x_test["Vinj"]
inj_ts = inj_ts.reset_index()
inj_ts

In [ ]:
test_sequence =[]
for i,seq in enumerate (x_test['Sequence']):
    test_sequence=np.append(test_sequence, seq)

#test_sequence
Xtest = encode_input(test_sequence, best_featvec)
Xtest = pd.concat([Xtest, inj_ts['Vinj']], axis=1)

for col in  Xtest.columns[0:30]:
    Xtest[col] = pd.to_numeric(Xtest[col], errors='coerce')

Xtest.head(10)

In [ ]:
Xtest.shape

In [ ]:
test_tRvec = y_test['tR1']
test_tRvec.shape

In [ ]:
print(isnullcolumns(Xtest))

In [ ]:
r = Xtest.index[np.isinf(Xtest).any(1)]
print(r)

In [ ]:
pred = best_estim.predict(Xtest)

rmse_err = skl_me.mean_squared_error(test_tRvec, pred, squared = False)

r2 = r2_score(test_tRvec, pred)

print (f'G2 - Best Features GB hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2)
print (f"RMSE: %.3f" % rmse_err)

In [ ]:
acc_test = round(best_estim.score(Xtest, test_tRvec), 3)
print(" r2", acc_test)

In [ ]:
plt.figure(figsize=(5,4))
plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

p1 = max(max(pred), max(test_tRvec))
p2 = min(min(pred), min(test_tRvec))
plt.plot([p1, p2], [p1, p2], 'k-')
plt.xlabel("$ Observed_ t_R (min)$", fontsize=11)
plt.ylabel(" $Predicted_ t_R (min)$", fontsize=11)
plt.axis('equal')
plt.savefig('G2_GB.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

### 2. Random Forest

In [ ]:
df_tRvec

In [ ]:
len(sequence)

In [ ]:
#setting the parameters for the RF model
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 300, num = 5)] # Number of trees in random forest
max_depth = [int(x) for x in np.linspace(5, 500, num = 10)]# Maximum number of levels in tree
min_samples_split = [2, 4]# Minimum number of samples required to split a node
min_samples_leaf = [1, 2]# Minimum number of samples required at each leaf node

model_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf
            }

In [ ]:
start = time.time()

#__defining variables
splits = 3
count = 0
best_perfomer_dict=dict()
scores = []

#__applying model to every feature combination, 31 features' combinations incl.  position and sulfur count features and excl. hairpin features

for j, featcomb in enumerate(featcombinations):
    inputvec_train = encode_input(sequence, featcomb)
    #inputvec_train['Vinj']=x_train["Vinj"]
    inputvec_train = pd.concat([inputvec_train, inj['Vinj']], axis=1)
    #print(inputvec_train)
    #print(isnullcolumns(inj['Vinj']))
    
    
    for col in  inputvec_train.columns[0:30]:
        inputvec_train[col] = pd.to_numeric(inputvec_train[col], errors='coerce')
#print (inputvec_train.info())
        
    xtrain, xvalid, ytrain, yvalid= train_test_split(inputvec_train, df_tRvec, test_size = 0.2, random_state = 42, shuffle=True)
        #print(len(X_train))
        #print(X_train)
        #print(X_test)
        
    model = RandomForestRegressor()
                
    grid = skl_ms.GridSearchCV(model, model_params, scoring = ['neg_root_mean_squared_error', 'r2'], refit = 'neg_root_mean_squared_error', cv = splits)
    grid.fit(xtrain, ytrain)
    model = grid.best_estimator_
                
    pre = model.predict(xvalid)
    err = skl_me.mean_squared_error(yvalid, pre, squared = False)
    count += 1
    print(f'\ncount: {count}')
            
    if ' '.join(featcomb) in best_perfomer_dict.keys():

        best_perfomer_dict[f' '.join(featcomb)] += err
            
    else:
            
        best_perfomer_dict[' '.join(featcomb)] = err
            

print(f'\n --- %s seconds --- % {time.time() - start}')

Min_error = min(best_perfomer_dict.values()) 
Min_key = min(best_perfomer_dict, key=best_perfomer_dict.get)

In [ ]:
model = grid.best_estimator_
print ("grid.best_estimator_", grid.best_estimator_)
print ("grid.best_score_", grid.best_score_)
        
filename = f"RF_G2.sav"
        
pickle.dump(model, open(filename, 'wb'))

In [ ]:
print ('Grid Search and Features selection results:\n')
print (best_perfomer_dict)

In [ ]:
print(f"features {Min_key}")
print(f"RMSE: {Min_error}")
print(f'Best parameters: {grid.best_estimator_}')

In [ ]:
RF_results = pd.DataFrame(grid.cv_results_)
RF_results.to_excel("G2_RF_grid.xlsx")

In [ ]:
best_estim = grid.best_estimator_
best_estim

In [ ]:
df_tRvec.shape

In [ ]:
best_featvec = ['count','sulfurcount' ,'scontact']

In [ ]:
alldata_train = encode_input(sequence, best_featvec)
alldata_train = pd.concat([alldata_train, inj['Vinj']], axis=1)

for col in  alldata_train.columns[0:30]:
    alldata_train[col] = pd.to_numeric(alldata_train[col], errors='coerce')

alldata_train

In [ ]:
alldata_train.info()

In [ ]:
best_estim.fit(alldata_train, df_tRvec)

In [ ]:
acc_train = round(best_estim.score(alldata_train, df_tRvec), 3)
print("training r2", acc_train)

In [ ]:
predtr = best_estim.predict(alldata_train)

rmsetr = skl_me.mean_squared_error(df_tRvec, predtr, squared = False)

print ('training rmse', rmsetr)

In [ ]:
x_test.head()

In [ ]:
inj_ts.shape

In [ ]:
test_sequence =[]
for i,seq in enumerate (x_test['Sequence']):
    test_sequence=np.append(test_sequence, seq)

#test_sequence
Xtest = encode_input(test_sequence, best_featvec)
Xtest = pd.concat([Xtest, inj_ts['Vinj']], axis=1)

for col in  Xtest.columns[0:30]:
    Xtest[col] = pd.to_numeric(Xtest[col], errors='coerce')

Xtest.head(5)

In [ ]:
Xtest.shape

In [ ]:
test_tRvec = y_test['tR1']
test_tRvec.shape

In [ ]:
r = Xtest.index[np.isinf(Xtest).any(1)]
print(r)

In [ ]:
print(isnullcolumns(Xtest))

In [ ]:
pred = best_estim.predict(Xtest)

rmse_err = skl_me.mean_squared_error(test_tRvec, pred, squared = False)

r2 = r2_score(test_tRvec, pred)

print (f'G2 - Best Features RF hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2)
print (f"RMSE: %.3f" % rmse_err)

In [ ]:
plt.figure(figsize=(5,4))
plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

p1 = max(max(pred), max(test_tRvec))
p2 = min(min(pred), min(test_tRvec))
plt.plot([p1, p2], [p1, p2], 'k-')
plt.xlabel("$ Observed_ t_R (min)$", fontsize=11)
plt.ylabel(" $Predicted_ t_R (min)$", fontsize=11)
plt.axis('equal')
plt.savefig('G2_RF.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

### 3. Decision tree

In [ ]:
params = {
    'max_depth' : [2,10,20,30,40,100],
    'min_samples_split' : [2,4,8], 
    'min_samples_leaf': [1,2,4,6,8]
}

In [ ]:
df_tRvec

In [ ]:
len(sequence)

In [ ]:
start = time.time()

#__defining variables
splits = 3
count = 0
best_perfomer_dict=dict()
scores = []

#__applying model to every feature combination, 31 features' combinations incl.  position and sulfur count features and excl. hairpin features

for j, featcomb in enumerate(featcombinations):
    inputvec_train = encode_input(sequence, featcomb)
    #inputvec_train['Vinj']=x_train["Vinj"]
    inputvec_train = pd.concat([inputvec_train, inj['Vinj']], axis=1)
    #print(inputvec_train)
    #print(isnullcolumns(inj['Vinj']))
    
    for col in  inputvec_train.columns[0:30]:
        inputvec_train[col] = pd.to_numeric(inputvec_train[col], errors='coerce')
        
    xtrain, xvalid, ytrain, yvalid= train_test_split(inputvec_train, df_tRvec, test_size = 0.2, random_state = 42, shuffle=True)
        #print(len(X_train))
        #print(X_train)
        #print(X_test)
        
    model =  DecisionTreeRegressor()
                
    grid = skl_ms.GridSearchCV(model, params, scoring = ['neg_root_mean_squared_error','r2'], refit = 'neg_root_mean_squared_error', cv = splits)
    grid.fit(xtrain, ytrain)
    model = grid.best_estimator_
                
    pre = model.predict(xvalid)
    err = skl_me.mean_squared_error(yvalid, pre, squared = False)
    count += 1
    print(f'\ncount: {count}')
            
    if ' '.join(featcomb) in best_perfomer_dict.keys():

        best_perfomer_dict[f' '.join(featcomb)] += err
            
    else:
            
        best_perfomer_dict[' '.join(featcomb)] = err
            

print(f'\n --- %s seconds --- % {time.time() - start}')

Min_error = min(best_perfomer_dict.values()) 
Min_key = min(best_perfomer_dict, key=best_perfomer_dict.get)

In [ ]:
G2_DT_grid = pd.DataFrame(grid.cv_results_)
G2_DT_grid.to_excel('G2_DT_grid.xlsx')

In [ ]:
print ('Grid Search and Features selection results:\n')
print (best_perfomer_dict)

In [ ]:
model = grid.best_estimator_
print ("grid.best_estimator_", grid.best_estimator_)
print ("grid.best_score_", grid.best_score_)
        
filename = f"DT_G2.sav"
        
pickle.dump(model, open(filename, 'wb'))

In [ ]:
print(f"features {Min_key}")
print(f"RMSE: {Min_error}")
print(f'Best parameters: {grid.best_estimator_}')

In [ ]:
best_estim = grid.best_estimator_
best_estim

In [ ]:
df_tRvec

In [ ]:
best_featvec=['count','position']

In [ ]:
pd.set_option('display.max_columns', None)

alldata_train = encode_input(sequence, best_featvec)
alldata_train = pd.concat([alldata_train, inj['Vinj']], axis=1)

for col in  alldata_train.columns[0:30]:
    alldata_train[col] = pd.to_numeric(alldata_train[col], errors='coerce')

alldata_train

In [ ]:
best_estim.fit(alldata_train, df_tRvec)

In [ ]:
acc_train = round(best_estim.score(alldata_train, df_tRvec), 3)
print("training r2", acc_train)

In [ ]:
predtr = best_estim.predict(alldata_train)

rmsetr = skl_me.mean_squared_error(df_tRvec, predtr, squared = False)

print (rmsetr)

In [ ]:
test_sequence =[]
for i,seq in enumerate (x_test['Sequence']):
    test_sequence=np.append(test_sequence, seq)

#test_sequence
Xtest = encode_input(test_sequence, best_featvec)
Xtest = pd.concat([Xtest, inj_ts['Vinj']], axis=1)

for col in  Xtest.columns[0:30]:
    Xtest[col] = pd.to_numeric(Xtest[col], errors='coerce')

Xtest.head(5)

In [ ]:
Xtest.shape

In [ ]:
test_tRvec = y_test['tR1']
test_tRvec.shape

In [ ]:
r = Xtest.index[np.isinf(Xtest).any(1)]
print(r)

In [ ]:
print(isnullcolumns(Xtest))

In [ ]:
pred = best_estim.predict(Xtest)

rmse_err = skl_me.mean_squared_error(test_tRvec, pred, squared = False)

r2 = r2_score(test_tRvec, pred)

print (f'G2 - Best Features DT hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2)
print (f"RMSE: %.3f" % rmse_err)

In [ ]:
plt.figure(figsize=(5,4))
plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

p1 = max(max(pred), max(test_tRvec))
p2 = min(min(pred), min(test_tRvec))
plt.plot([p1, p2], [p1, p2], 'k-')
plt.xlabel("$ Observed_ t_R (min)$", fontsize=11)
plt.ylabel(" $Predicted_ t_R (min)$", fontsize=11)
plt.axis('equal')
plt.savefig('G2_DT.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

### SVR

In [ ]:
len(sequence)

In [ ]:
df_tRvec

In [ ]:
params = {'C' : np.linspace(1,1000, 50), 
              'gamma': [0.1, 0.01, 0.001],
              'epsilon': np.linspace(0.0001, 0.0002, 10),
              'kernel': ['rbf']
         }

In [ ]:
start = time.time()

#__defining variables
splits = 3
count = 0
best_perfomer_dict=dict()
scores = []

for j, featcomb in enumerate(featcombinations):
    inputvec_train = encode_input(sequence, featcomb)
    #inputvec_train['Vinj']=x_train["Vinj"]
    inputvec_train = pd.concat([inputvec_train, inj], axis=1)
    #print(inputvec_train)
    #print(isnullcolumns(inj['Vinj']))
    
#print(inputvec_train.info())

    for col in  inputvec_train.columns[0:30]:
        inputvec_train[col] = pd.to_numeric(inputvec_train[col], errors='coerce')
        
    xtrain, xvalid, ytrain, yvalid= train_test_split(inputvec_train, df_tRvec, test_size = 0.2, random_state = 42, shuffle=True)
        
    model = svm.SVR()
                
    grid = skl_ms.GridSearchCV(model, params, scoring = ['neg_root_mean_squared_error','r2'], refit = 'neg_root_mean_squared_error', cv = splits)
    grid.fit(xtrain, ytrain)
    model = grid.best_estimator_
                
    pre = model.predict(xvalid)
    err = skl_me.mean_squared_error(yvalid, pre, squared = False)
    count += 1
    print(f'\ncount: {count}')
            
    if ' '.join(featcomb) in best_perfomer_dict.keys():

        best_perfomer_dict[f' '.join(featcomb)] += err
            
    else:
            
        best_perfomer_dict[' '.join(featcomb)] = err
            

print(f'\n --- %s seconds --- % {time.time() - start}')

Min_error = min(best_perfomer_dict.values())  
Min_key = min(best_perfomer_dict, key=best_perfomer_dict.get)

In [ ]:
best_perfomer_dict

In [ ]:
print(f"features {Min_key}")
print(f"RMSE: {Min_error}")
print(f'Best parameters: {grid.best_estimator_}')

In [ ]:
best_estim = grid.best_estimator_
print ("grid.best_estimator_", grid.best_estimator_)
print ("grid.best_score_", grid.best_score_)
        
filename = f"SVR_G2.sav"
        
pickle.dump(best_estim, open(filename, 'wb'))

In [ ]:
best_featvec = ['count','sulfurcount']

In [ ]:
len(sequence)

In [ ]:
pd.set_option('display.max_columns', None)

alldata_train = encode_input(sequence, best_featvec)
alldata_train = pd.concat([alldata_train, inj], axis=1)

for col in  alldata_train.columns[0:30]:
    alldata_train[col] = pd.to_numeric(alldata_train[col], errors='coerce')

alldata_train

In [ ]:
best_estim

In [ ]:
df_tRvec.shape

In [ ]:
best_estim.fit(alldata_train, df_tRvec)

In [ ]:
acc_train = round(best_estim.score(alldata_train, df_tRvec), 3)
print("training r2", acc_train)

In [ ]:
predtr = best_estim.predict(alldata_train)

rmsetr = skl_me.mean_squared_error(df_tRvec, predtr, squared = False)

print (rmsetr)

In [ ]:
x_test = pd.read_excel('G22_xtest.xlsx')
y_test = pd.read_excel('G22_ytest.xlsx')

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
x_test['Sequence'].shape

In [ ]:
inj_ts = x_test["Vinj"]
#inj_ts = inj_ts.reset_index()
len(inj_ts)

In [ ]:
inj_ts

In [ ]:
test_sequence =[]
for i,seq in enumerate (x_test['Sequence']):
    test_sequence=np.append(test_sequence, seq)

#test_sequence
Xtest = encode_input(test_sequence, best_featvec)
Xtest = pd.concat([Xtest, inj_ts], axis=1)

for col in  Xtest.columns[0:30]:
    Xtest[col] = pd.to_numeric(Xtest[col], errors='coerce')

Xtest.head(5)

In [ ]:
print(isnullcolumns(Xtest))

In [ ]:
test_tRvec = y_test['tR1']
test_tRvec.shape

In [ ]:
test_tRvec

In [ ]:
pred = best_estim.predict(Xtest)

rmse_err = skl_me.mean_squared_error(test_tRvec, pred, squared = False)

r2 = r2_score(test_tRvec, pred)

print (f'G2 - Best Features SVR hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2)
print (f"RMSE: %.3f" % rmse_err)

In [ ]:
plt.figure(figsize=(5,4))
plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

p1 = max(max(pred), max(test_tRvec))
p2 = min(min(pred), min(test_tRvec))
plt.plot([p1, p2], [p1, p2], 'k-')
plt.xlabel("$ Observed_ t_R (min)$", fontsize=11)
plt.ylabel(" $Predicted_ t_R (min)$", fontsize=11)
plt.axis('equal')
plt.savefig('G2_SVR.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()